<a href="https://colab.research.google.com/github/RyanChen12035/w266-NLP/blob/main/w266_final_project_model3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
"""
Sentiment analysis of movie review
1. Use GPT2, Decoder based, pretrained with EN text and has 12 layers of transformer layers. On top of it is a projection layer with shape(768, 50257) and softmax layer
2. We are not making use of the autoregressive nature of GPT, only get the last token as output and feed it to the classification head to do the task.
3. The FFN consists of two linear transformations with weights of shapes (768, 3072) and (3072, 768)
4. No CLS token in decoder based model, conventionally, we pick up the last token of each output Nth (, embedding dimension)
5. Each layer of transformer would pass down output shape (batch, seq, embedding dimension), and we can pick up the last token and get   to know how the representation evolved.
6. For the intermediate weights of FFN of each layers of transformer, it has representation shape of  (,768) and we can also put it into the comparison to the last token of the final transformer layer.

"""

"\nSentiment analysis of movie review\n1. Use GPT2-medium, Decoder based, pretrained with EN text and has 24 layers of transformer layers. On top of it is a projection layer with shape(768, 50257) and softmax layer\n2. It was pretrained by EN text, so basically it can not handle EN to CN task without fine-tuning\n3. Need a tokenizer that incorperate EN and CN, the embedding size and dictionary size are (768, 119,547)\n4. Fine-tune, may need prompt + EN,CN pair.\n5. The FFN consists of two linear transformations with weights of shapes (768, 3072) and (3072, 768)\n6. Don't need mask in decoder only model\n7. No CLS token in decoder based model, conventionally, we pick up the last token of each output Nth (, embedding dimension) so if the intput lenghth is 10 runs (assuming we have 10 input length, prompt (EN) + ZH tokens + newly generated EN tokens),\n   and the maximum length is 20, we would go through the decoder model iteratively 10 more times or until meets the EOS token.\n8. Each la

In [1]:
!pip install pydot --quiet
!pip install tensorflow-datasets --quiet
!pip install transformers --quiet

In [2]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [3]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-13f2d3b4-f6d0-1763-475b-2b3d9a167098)


In [4]:
import os
import time
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Embedding, Input, Dense, Lambda, Dropout, Conv1D, GlobalMaxPooling1D, Concatenate, Activation
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K
import tensorflow_datasets as tfds
from transformers import GPT2Tokenizer, TFGPT2LMHeadModel, GPT2Config
import sklearn as sk
import os
from nltk.data import find
import matplotlib.pyplot as plt
import re
import tensorflow as tf

In [5]:
train_data, test_data = tfds.load(
    name="imdb_reviews",
    split=('train[:80%]', 'test[80%:]'),
    as_supervised=True)

train_examples, train_labels = next(iter(train_data.batch(20000)))
test_examples, test_labels = next(iter(test_data.batch(5000)))

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteK5LGYW/imdb_reviews-train.tfrecord…

Generating test examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteK5LGYW/imdb_reviews-test.tfrecord*…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteK5LGYW/imdb_reviews-unsupervised.t…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [15]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

config = GPT2Config.from_pretrained("gpt2", output_hidden_states=True)
gpt2_model = TFGPT2LMHeadModel.from_pretrained("gpt2", config=config)
tokenizer.pad_token = tokenizer.eos_token
MAX_SEQUENCE_LENGTH = 100

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

In [7]:
#Embedding size of GPT2 tokenizer: 768
#Dictionary size of GPT2 tokenizer: 50257


train_examples_str = [x.decode('utf-8') for x in train_examples.numpy()]
test_examples_str = [x.decode('utf-8') for x in test_examples.numpy()]

#training data
gpt_train_tokenized = tokenizer(train_examples_str,
              max_length=MAX_SEQUENCE_LENGTH,
              truncation=True,
              padding='max_length',
              return_tensors='tf')

gpt_train_inputs = {
    'input_ids': gpt_train_tokenized['input_ids'],
    'attention_mask': gpt_train_tokenized['attention_mask']
}

gpt_train_labels = np.array(train_labels)


#validation data
gpt_val_tokenized = tokenizer(test_examples_str,
              max_length=MAX_SEQUENCE_LENGTH,
              truncation=True,
              padding='max_length',
              return_tensors='tf')

gpt_val_inputs = {
    'input_ids': gpt_val_tokenized['input_ids'],
    'attention_mask': gpt_val_tokenized['attention_mask']
}

gpt_val_labels = np.array(test_labels)

In [16]:
gpt_train_inputs['input_ids']

<tf.Tensor: shape=(20000, 100), dtype=int32, numpy=
array([[ 1212,   373,   281, ...,   373,  2147,   475],
       [   40,   423,   587, ...,   262,  3741,   286],
       [   44,  1236, 12566, ...,   319,   262,  5398],
       ...,
       [11380,    11,   314, ...,    33, 12599,   318],
       [ 7003,   314, 13178, ...,  7110,  1627,  1838],
       [ 1212, 35214,  2646, ..., 18989,   510,   355]], dtype=int32)>

In [17]:
gpt_train_labels

array([0, 0, 0, ..., 0, 0, 0])

In [8]:
#24 layers of transformer
#A drop out layer + dense layer with 100 hidden layer size on top + final layer with sigmoid as activation function


def create_gpt_last_model(gpt_model,
                          max_sequence_length=MAX_SEQUENCE_LENGTH,
                          hidden_size = 100,
                          dropout=0.3,
                          learning_rate=0.00005):
    """
    Build a simple classification model with gpt. Use the last token output for classification purposes.
    """

    gpt_model.trainable = True #True

    #input layers of gpt, shape (batch, max_sequence_length), model will be fit with gpt_train_inputs
    input_ids = Input(shape=(max_sequence_length,), dtype=tf.int32, name='input_ids')
    attention_mask = Input(shape=(max_sequence_length,), dtype=tf.int32, name='attention_mask')

    # GPT-2 model
    #model.generate() for iteratively generating (autoregressive)
    #we only do it one time.
    gpt2_outputs = gpt_model(input_ids=input_ids, attention_mask=attention_mask)

    # Use the last hidden state of the last transformer layer for classification, ingore linear layer and softmax layer
    # Select the last token of hidden state
    last_hidden_state_last_token = gpt2_outputs.logits[:, -1, :]

    #Add a dropout layer
    x = Dropout(dropout)(last_hidden_state_last_token)

    #Add a fully connected layer for classification
    x = Dense(hidden_size, activation='relu')(x)

    #Final output layer for classification, assuming it's binary task
    output = Dense(1, activation='sigmoid')(x)


    # Create the model
    classification_model = Model(inputs=[input_ids, attention_mask], outputs=output)


    #Model complie
    classification_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                                 loss='binary_crossentropy',
                                 metrics=['accuracy'])

    return classification_model


In [9]:
model = create_gpt_last_model(gpt2_model)

history = model.fit(gpt_train_inputs,
                    gpt_train_labels,
                    epochs=2, #2
                    batch_size=8,
                    validation_data=(gpt_val_inputs, gpt_val_labels))

2500/2500 [==============================] - 1928s 746ms/step - loss: 1.6376 - accuracy: 0.7498 - val_loss: 0.3231 - val_accuracy: 0.8710


In [10]:
# Example test reviews
"""
1. Identifying Emotional Tone
Sub-Task: Determine the emotional tone of the review (e.g., positive, negative, neutral).
Test Reviews:
    EX1 "The movie's breathtaking scenery and exceptional soundtrack added depth to its rich storytelling." -> Positive Tone
    EX2 "The film was a letdown with its lackluster plot and uninspired performances." -> Negative Tone
2. Analyzing Subjective Statements
Sub-Task: Detect subjective statements or opinions in the review.
Test Reviews:
    EX3 "In my opinion, the film's portrayal of historical events was highly inaccurate." -> Subjective
    EX4 "The movie won three Academy Awards, including Best Picture." -> Objective
3. Evaluating Specific Aspects (Acting, Plot, Cinematography)
Sub-Task: Assess specific aspects of the movie like acting quality, plot development, and cinematography.
Test Reviews:
    EX5 "The acting was superb, with each character bringing depth and emotion to the screen." -> Positive Acting
    EX6 "The plot was predictable and lacked originality, making the movie quite boring." -> Negative Plot
4. Recognizing Extremes in Sentiment
Sub-Task: Identify reviews with extremely positive or negative sentiments.
Test Reviews:
    EX7 "This is possibly the worst movie ever made, with no redeeming qualities whatsoever." -> Extremely Negative
    EX8 "An absolute masterpiece, every moment was captivating and a joy to watch." -> Extremely Positive
5. Detecting Sarcasm or Irony
Sub-Task: Detect sarcasm or irony, which can often invert the apparent sentiment of a statement.
Test Reviews:
    EX9 "Oh great, another predictable rom-com, just what the world needs." -> Sarcasm
    EX10 "I loved how the movie ended abruptly without resolving any plot points." -> Irony
"""

# First reivews is positive tone and the second is negative tone
test_reviews = [
    "In my opinion, the film's portrayal of historical events was highly inaccurate.",
    "The movie won three Academy Awards, including Best Picture.",
    "In my opinion, the film's portrayal of historical events was highly inaccurate.",
    "The movie won three Academy Awards, including Best Picture.",
    "The acting was superb, with each character bringing depth and emotion to the screen.",
    "The plot was predictable and lacked originality, making the movie quite boring.",
    "This is possibly the worst movie ever made, with no redeeming qualities whatsoever.",
    "An absolute masterpiece, every moment was captivating and a joy to watch.",
    "Oh great, another predictable rom-com, just what the world needs.",
    "I loved how the movie ended abruptly without resolving any plot points."
]

In [11]:
test_reviews[0]

"In my opinion, the film's portrayal of historical events was highly inaccurate."

In [16]:
def extract_transformer_outputs(model, tokenizer, text, max_length=MAX_SEQUENCE_LENGTH):
    # Tokenize the input text
    inputs = tokenizer(text, return_tensors="tf", max_length=max_length, padding=True, truncation=True)

    # Get outputs from the model
    outputs = model(**inputs, output_hidden_states=True)

    # Extract the hidden states from each layer
    hidden_states = outputs.hidden_states

    return hidden_states

# Example usage
hidden_states = extract_transformer_outputs(gpt2_model, tokenizer, test_reviews[0])
print(hidden_states)
print(len(hidden_states))

(<tf.Tensor: shape=(1, 15, 768), dtype=float32, numpy=
array([[[-0.02854068, -0.2792984 ,  0.1108078 , ...,  0.1995235 ,
         -0.03746982, -0.00815885],
        [ 0.1817398 ,  0.05527091, -0.02113888, ...,  0.08475073,
          0.11588293,  0.02794035],
        [ 0.1682339 , -0.00162578,  0.03334213, ..., -0.13172291,
          0.01747025, -0.08828945],
        ...,
        [ 0.06265579,  0.13509348,  0.20024075, ...,  0.2057868 ,
          0.02212108,  0.05937694],
        [-0.00805705,  0.08783852,  0.19446757, ..., -0.14721513,
          0.02029728,  0.08548182],
        [ 0.04718417,  0.01130205,  0.12396982, ..., -0.07390399,
          0.04637358,  0.09471334]]], dtype=float32)>, <tf.Tensor: shape=(1, 15, 768), dtype=float32, numpy=
array([[[ 0.8681928 , -1.3534713 ,  0.57987463, ..., -1.6423428 ,
         -0.5011883 ,  0.33389375],
        [-1.0500813 , -0.30427533,  0.02456912, ...,  0.72792625,
          1.7174268 , -0.3371492 ],
        [ 0.1120162 ,  0.11955559, -1.84004

In [ ]:
hidden_states[0]

In [17]:
import numpy as np
from numpy.linalg import norm

def calculate_similarities(hidden_states):
    # Number of examples and layers
    num_examples, num_layers = hidden_states[0].shape[0], len(hidden_states)

    # Initializing arrays to store the results
    cosine_similarities = np.zeros((num_examples, num_layers))
    dot_products = np.zeros((num_examples, num_layers))

    # Final layer's CLS output
    final_layer_output = cls_outputs[-1]

    # Calculating similarities and dot products
    for i in range(num_layers):
        for j in range(num_examples):
            # Extracting the CLS output for the current layer and example
            current_output = cls_outputs[i][j]

            # Cosine Similarity
            cosine_similarities[j, i] = np.dot(current_output, final_layer_output[j]) / (norm(current_output) * norm(final_layer_output[j]))

            # Dot Product
            dot_products[j, i] = np.dot(current_output, final_layer_output[j])

    return cosine_similarities, dot_products


simlarity = calculate_similarities(cls_hidden_states)

13